# Module: do_aperture_photometry

This function uses the photutils packages to do aperture photometry of the aligned & reduced data.

INPUT:

     Filename of the the data image and mean FWHM.
     
OUTPUT:

     Magnitude of sources of different apertures and their corresponding errors.

In [ ]:
def do_aperture_photometry(filename, fwhm):

Now select a mode for input images.

In [ ]:
choice=input("Enter the mode. Please use the keywords\n 'single' for single image, 'multiple'  for multiple images: \n\n")

if (choice=='single'):
    print('single mode')
elif (choice ==' multiple'):
    print(' multiple image mode')
else:
    print('Enter valid choice!!!')

Now    
       1. Identify the sources, their position. 
       2. Create the multiple apertures around the sources.
       3. Get the magnitude and magnitude error for different apertures.

In [ ]:
data,header=fits.getdata(filename,header=True)
data=2*data
exposure=header['EXPOSURE']
print('Exposure is',exposure)

sigma_clip = SigmaClip(sigma=3., maxiters=10)
bkg_estimator = MedianBackground()
bkg = Background2D(data, (10,10), filter_size=(3, 3),sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
back=bkg.background # this is the background we need for the background subtraction.
back2=np.median(bkg.background)


mask = data == 0
unit = u.electron / u.s


xdf_image = CCDData(data, unit=unit, meta=header, mask=mask)
norm_image = ImageNormalize(vmin=1e-4, vmax=5e-2, stretch=LogStretch(), clip=False)
xdf_image_clipped = np.clip(xdf_image, 1e-4, None)

mean, median, std = sigma_clipped_stats(xdf_image.data, sigma=3.0, maxiters=20, mask=xdf_image.mask)

print('Finding the sources')

daofind = DAOStarFinder(fwhm=fwhm, threshold=10*std) # 3 sigma above the background.
sources = daofind(data-back)

#sources_findpeaks = find_peaks(xdf_image.data, mask=xdf_image.mask, threshold=30.*std, box_size=30, centroid_func=centroid_2dg)

print('We have found:',len(sources),' sources')
#print(sources)

#print(sources['xcentroid'], sources['ycentroid'],sources['fwhm'])
#positions=sources['xcentroid'], sources['ycentroid']
positions=np.genfromtxt('co_ordinates_list.txt',unpack=True,usecols=(0,1))
#print(positions)
radii=[ fwhm,2*fwhm, 3*fwhm]

#positions=(sources['xcentroid'], sources['ycentroid'])
apertures = [CircularAperture(positions, r=r) for r in radii]

an_ap = CircularAnnulus(positions, r_in=5*fwhm, r_out=7*fwhm)
#apers = [apertures, annulus_apertures]


#bkg_sigma=mad_std(data)
effective_gain=exposure
error=calc_total_error(data,back,effective_gain)


#error=0.1*data
phot_table = aperture_photometry(data-back, apertures,error=error)
phot_table2=aperture_photometry(data-back,an_ap)


bkg_mean = phot_table2['aperture_sum'] / an_ap.area()
bkg_sum = bkg_mean * an_ap.area()


final_sum0=phot_table['aperture_sum_0']-bkg_sum
final_sum1=phot_table['aperture_sum_1']-bkg_sum
final_sum2=phot_table['aperture_sum_2']-bkg_sum


mag_back=-2.5*np.log10(bkg_mean/exposure)+25
mag_0=-2.5*np.log10(final_sum0/exposure)+25
mag_1=-2.5*np.log10(final_sum1/exposure)+25
mag_2=-2.5*np.log10(final_sum2/exposure)+25

flux_err_0=phot_table['aperture_sum_err_0']
mag_err_0=1.09*flux_err_0/final_sum0

flux_err_1=phot_table['aperture_sum_err_1']
mag_err_1=1.09*flux_err_1/final_sum1

flux_err_2=phot_table['aperture_sum_err_2']
mag_err_2=1.09*flux_err_2/final_sum2

fig=plt.figure()
plt.imshow(data,cmap='gray',origin='lower',vmin=500,vmax=900)
colors=['red','green','yellow']
for i in range(len(apertures)):
    apertures[i].plot(color=colors[i], alpha=0.7)

an_ap.plot(color='green', alpha=0.7)
plt.show()

with open ('results.dat','w') as r:
    for i in range (len(phot_table)):
        #print(final_sum0[i],final_sum1[i],final_sum2[i],final_sum3[i],final_sum4[i],final_sum5[i],file=r)
        print(mag_back[i],mag_0[i],mag_err_0[i],mag_1[i],mag_err_1[i],mag_2[i],mag_err_2[i],file=r)


Photometry is completed & now we are ready to plot the light curve...!!

# Module: plot_light_curve

This function is used to plot the the light curve.

In [ ]:
def plot_light_curve(filename,list):

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~for plotting purposes~~~~~~~~#
    mp.rcParams['font.family']='serif'
    mp.rcParams['xtick.major.size']=10
    mp.rcParams['xtick.major.width']=2
    mp.rcParams['xtick.minor.size']=7
    mp.rcParams['xtick.minor.width']=2
    mp.rcParams['ytick.major.size']=10
    mp.rcParams['ytick.major.width']=2
    mp.rcParams['ytick.minor.size']=7
    mp.rcParams['ytick.minor.width']=2
    mp.rcParams['axes.linewidth']=1.5
    mp.rcParams['xtick.labelsize']=36
    mp.rcParams['ytick.labelsize']=36
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    filelist=list
    files=sorted(glob(filelist))
    julian_date=np.zeros(len(files))
    for i in range(0,len(files)):
        data=fits.open(files[i])
        header=data[0].header
        julian_date[i]=header['JD']

    # This is a dummy for flux
    julian_date-=2458870
    fluxd=np.zeros(len(files))
    for i in range(len(files)):
        fluxd[i]=random.randint(5,8)

    flux_errd=0.05*fluxd
    #################

    fig,ax=plt.subplots()
    plt.plot(julian_date,fluxd,'ko',markersize=25,label='I band light curve')
    plt.errorbar(julian_date,fluxd,yerr=flux_errd,capsize=5,fmt=' ')
    plt.ylabel("Magnitudes (instrumental)",fontsize=36)
    plt.xlabel("Julian Dates (2458870+)",fontsize=36)
    #plt.title("Deviation in lag values with respect to the variability",fontsize=36)
    ax.tick_params(axis="both",which='minor',direction="in")
    ax.tick_params(axis="both",which='major',direction="in")
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')
    ax.minorticks_on()
    ax.legend(fontsize=30)
    plt.show()
